Making Basic Chatbot with conversational History

In [1]:
import os

from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

Using GROQ library to use llama model

In [ ]:
from langchain_groq import ChatGroq
model = ChatGroq(model= "llama3.2", groq_api_key=groq_api_key)


In [5]:
from langchain_core.messages import HumanMessage, AIMessage
model.invoke([HumanMessage(content=" Hy My Name is Muhammad Awais and I am AI Engineer."),
              AIMessage(content="Hi Muhammad Awais. As an Ai i Engineer, How i can help you?"),
              HumanMessage(content="Hi What's my name and what do i do?")
              
              ]
             )


NameError: name 'model' is not defined

In [ ]:
from langchain_community import chat_message_histories
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
store = {}

def get_session_history(session_id:str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]= chat_message_histories
        return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)


In [1]:
config = {"configurable": {"session_id": "chat1"}}

In [2]:
response = with_message_history.invoke(
    [HumanMessage(content="Hy I am Muhammad Awais. I am AI Engineer."),
     
     ],
     config=config
)
response.content

NameError: name 'with_message_history' is not defined

In [ ]:
with_message_history.invoke(
    [
        HumanMessage(content="What's my name?")
    ],
    config=config
)

Same session id for remember chat History

In [3]:
# If i use different session id, it will not able to respond according to my chat histroy.
config_1 = {"configurable": {"session_id": "chat_1"}}

In [ ]:
#Now my app will not able to remember Chat  History

with_message_history.invoke(
    [
        HumanMessage(content="What's my naeme?")
    ],
    config=config_1
)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    "system", "You are a helpful assitent, Aswer the questions according to your nest ability",
    MessagesPlaceholder(variable_name="messages")
]
)

chain = prompt | model

response = chain.invoke({"messages": [HumanMessage(content= "Hi My name is Muhammad Awais")]})



In [ ]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

config_2 = {"configurable": {"session_id": "chat_3"}}

response = with_message_history.invoke([
    HumanMessage(content= "Hi My name is Muhammad Awais")
],
config=config_2)
response 

In [ ]:
answer = with_message_history.invoke([
    HumanMessage(content= "What's my name?")
],
config=config_2)
answer.content

In [ ]:
# Adding More Complexity
prompt = ChatPromptTemplate.from_messages([
    "system", "Act as a Language Translator, Transalte the input into {language}",
    MessagesPlaceholder(variable_name= "messages")]
)

chain = prompt | model

response = chain.invoke({"messages": [MessagesPlaceholder(content= "hi My name is mUhammad Awais")], "language": "english"})


In [ ]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history, input_messages_key="messages")

In [ ]:
config_3 = {"configurable": {"session_id": "chat3"}}

response = with_message_history.invoke({
    "messages": [HumanMessage(content=" Hy My name is muhammad Awais")],
    "language": "english"

}, config=config_3)

response.content

In [ ]:
response = with_message_history.invoke({
    "messages": [HumanMessage(content="What's my name?")], "language": "english"
}, config=config_3)

In [2]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens = 70,
    strategy = "last",
    token_counter = model,
    include_system = True,
    allow_partial = False,
    start_on = "human"
)
messages = [
    SystemMessage(content="You are a good assitant"),
    HumanMessage( content= "Hi I am Bob"),
    AIMessage(content= "hi"),
    HumanMessage(content="I like Venilla Cream"),

    AIMessage(content= " nice"),
    HumanMessage(content= "What is 2 + 2?"),
    AIMessage(content="4")

]
trimmer.invoke(messages)



NameError: name 'model' is not defined

### How to pass Trimmer to chain


In [ ]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = ( RunnablePassthrough.assign(messages= itemgetter("messages") | trimmer) | prompt | model)


In [ ]:
chain.invoke({
    "messages": messages + [HumanMessage(content= "What ice cream do i like?")],
    "language": "English"
})

response.content


In [ ]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key= "messages"
)
config_4 = {"configurable": {"session_id": "chat4"}}

In [ ]:
answer_1 =with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content= "What is my name?")],
        "language": "English"
    },
    config=config_4
)
answer_1.content



In [ ]:
from langchain_core.documents import Document
documents = [
    Document(page_content= "Dogs are great campanion know for their loylty and freindlines behaviour.",
             metadata = {"sourse": "memal-pets-docs"}),
    Document(page_content="Cats are independent pets they ofetn enjoy their own space.",
             metadata = {"source": "memal-pets-docs"}),
    Document(page_content="Goldfish is popular pet for bignners, requiring relatively simple care.",
             metadata = {"sourse": "memal-pets-docs"})]

documents